In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#from joblib import dump, loada
import joblib

In [19]:
# Carga datos y crea variables dummy
df = pd.read_csv('juadores_fantasy.csv.csv')
# Guarda nombre original de columna
pos_col = 'posicion'
#categorical_cols = ['posicion', 'equipo']  
#df = pd.get_dummies(df, columns=categorical_cols)

In [20]:
# Codifica posicion y equipo a variables dummy
df = pd.get_dummies(df, columns=['posicion', 'equipo'])

In [21]:
# Obtén los nombres de las columnas dummy creadas
pos_dummies = [c for c in df.columns if c.startswith('posicion_')]

In [22]:
# Solicita entrada de posición
posicion = "delantero"


In [23]:
# Filtra usando una de las columnas dummy
df_filtrado = df[df[pos_dummies[0]] == 1]

In [24]:
# Separa las features de las variables a predecir  
X = df.drop(['Nombre', 'Total_puntos', 'Precio'], axis=1)
y_puntos = df['Total_puntos'] 
y_precio = df['Precio']

In [25]:
# Divide los datos en entrenamiento y prueba
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.3, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.3, random_state=42)

In [26]:
# Entrena un modelo para predecir puntos
rf_puntos = RandomForestRegressor()
rf_puntos.fit(X_train, y_puntos_train)

RandomForestRegressor()

In [27]:
# Entrena un modelo para predecir precio 
rf_precio = RandomForestRegressor(n_estimators=100)
rf_precio.fit(X_train, y_precio_train)

RandomForestRegressor()

In [28]:
# Realiza predicciones en los datos de test
y_puntos_pred = rf_puntos.predict(X_test)
y_precio_pred = rf_precio.predict(X_test)

In [29]:
# Evalúa los modelos
print("Puntos R2 Score:", rf_puntos.score(X_test, y_puntos_test))
mse = mean_squared_error(y_puntos_test, y_puntos_pred) 
print("Puntos MSE:", mse)

print("Precio R2 Score:", rf_precio.score(X_test, y_precio_test))
mse = mean_squared_error(y_precio_test, y_precio_pred)
print("Precio MSE:", mse)

Puntos R2 Score: 0.7214434821998522
Puntos MSE: 343.10239349928685
Precio R2 Score: 0.6769277293850464
Precio MSE: 11719542024337.43


In [30]:
# Obtén las predicciones ordenadas para encontrar los mayores
indices_puntos = y_puntos_pred.argsort()[::-1]
indices_precio = y_precio_pred.argsort()[::-1]

In [33]:
print(f"\n2 jugadores {posicion} con mayor puntaje predicho:")
for i in indices_puntos[:2]:
    nombre_jugador = df.loc[i, 'Nombre']
    puntaje_predicho = y_puntos_pred[i]
    posicion_jugador = df.loc[i, 'posicion']
    
    print(f"{nombre_jugador} ({posicion_jugador}): {puntaje_predicho}")


2 jugadores delantero con mayor puntaje predicho:


KeyError: 'posicion'

In [ ]:
print(f"\n2 jugadores {posicion} con mayor precio predicho:")
for i in indices_precio[:2]:
  print(df.loc[i,'Nombre'], y_precio_pred[i])


2 jugadores delantero con mayor precio predicho:
R. Alcaraz 19626610.0
R. Lewandowski 17576230.0


In [ ]:
# Crear un DataFrame con los resultados
resultados = pd.DataFrame({
    'Nombre': [df.loc[i,'Nombre'] for i in indices_puntos[:2]] + [df.loc[i,'Nombre'] for i in indices_precio[:2]],
    'Puntos_Predichos': list(y_puntos_pred[indices_puntos[:2]]) + [None, None],
    'Precio_Predicho': [None, None] + list(y_precio_pred[indices_precio[:2]])
})

# Guardar el DataFrame en un archivo CSV
resultados.to_csv('sugerencia.csv', index=False)

In [ ]:
joblib.dump(rf_puntos, 'random_forest.pkl')

['random_forest.pkl']